<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/CBF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Menggunakan Tutorial dari ChatGPT 4 : Kode Python untuk Sistem Rekomendasi MCRS dengan CBF

In [63]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
import nltk

# Mengunduh stopwords Bahasa Indonesia dari NLTK
nltk.download('stopwords')
indonesian_stopwords = stopwords.words('indonesian')

# Contoh data tentang destinasi wisata di Indonesia
data = {
    'Destination': ['Pantai Kuta', 'Borobudur', 'Pantai Sanur', 'Gunung Bromo', 'Raja Ampat'],
    'Description': [
        'Pantai Kuta terkenal dengan pasir putihnya yang indah dan ombaknya yang cocok untuk berselancar.',
        'Borobudur adalah candi Buddha yang terkenal di dunia, berlokasi di Magelang, Jawa Tengah.',
        'Pantai Sanur terletak di Bali, dikenal dengan sunrise yang menakjubkan dan suasana yang tenang.',
        'Gunung Bromo adalah gunung berapi aktif di Jawa Timur, terkenal dengan pemandangan matahari terbitnya.',
        'Raja Ampat, yang berlokasi di Papua Barat, adalah surga bagi penyelam dan dianggap salah satu tempat menyelam terbaik di dunia.'
    ],
    'Rating': [4.0, 4.5, 4.2, 4.7, 4.8],
    'Popularity': [5000, 8000, 3000, 4000, 6000]
}

df = pd.DataFrame(data)

# Vektorisasi deskripsi menggunakan TF-IDF dan stop words Indonesia
tfidf = TfidfVectorizer(stop_words=indonesian_stopwords)
tfidf_matrix = tfidf.fit_transform(df['Description'])

# Normalisasi kolom Rating dan Popularity
scaler = MinMaxScaler()
df[['Rating', 'Popularity']] = scaler.fit_transform(df[['Rating', 'Popularity']])

# Bobot untuk masing-masing kriteria
weights = {'Description': 0.5, 'Rating': 0.25, 'Popularity': 0.25}

# Fungsi untuk mencari rekomendasi berdasarkan kata kunci
def get_recommendations(keyword):
    # Vektorisasi kata kunci
    keyword_vector = tfidf.transform([keyword])

    # Hitung skor kesamaan cosine
    cosine_sim = np.dot(keyword_vector, tfidf_matrix.T).toarray()

    # Hitung skor akhir dengan mempertimbangkan rating dan popularitas
    final_scores = cosine_sim.flatten() * weights['Description'] + df['Rating'] * weights['Rating'] + df['Popularity'] * weights['Popularity']

    # Mendapatkan indeks dari skor tertinggi
    top_idx = np.argmax(final_scores)

    return df.iloc[top_idx]['Destination']

# Contoh pemanggilan fungsi
keyword_input = "gunung"
recommended_destination = get_recommendations(keyword_input)
print(f"Destinasi yang direkomendasikan: {recommended_destination}")


Destinasi yang direkomendasikan: Gunung Bromo


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


# Menambahkan Evaluasi Hasil Berupa Confusion Matris, MAE dan RMSE

In [68]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import confusion_matrix, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler

# Contoh dataset (asumsikan memiliki kolom `destination`, `description`, `rating`, `price`, `popularity`)
data = {
    'destination': ['Bali', 'Lombok', 'Yogyakarta', 'Bandung', 'Raja Ampat'],
    'description': [
        'Pantai, budaya, pura, wisata air',
        'Pantai, gunung, snorkling, diving',
        'Sejarah, candi, kuliner, budaya',
        'Wisata alam, gunung, air terjun, kuliner',
        'Snorkling, pantai, keindahan alam, diving'
    ],
    'rating': [4.8, 4.5, 4.7, 4.3, 4.9],
    'price': [1000, 800, 700, 500, 1500],
    'popularity': [20000, 15000, 18000, 12000, 10000]
}

# Membuat DataFrame
df = pd.DataFrame(data)

# Normalisasi kriteria numerik (rating, harga, popularitas)
scaler = MinMaxScaler()
df[['rating_norm', 'price_norm', 'popularity_norm']] = scaler.fit_transform(df[['rating', 'price', 'popularity']])

# Bobot untuk masing-masing kriteria
weights = {'rating': 0.4, 'price': 0.2, 'popularity': 0.4}

# Menghitung skor akhir berdasarkan bobot
df['final_score'] = (
    weights['rating'] * df['rating_norm'] +
    weights['price'] * (1 - df['price_norm']) +  # Harga lebih murah lebih baik
    weights['popularity'] * df['popularity_norm']
)

# Vektorisasi teks menggunakan TF-IDF dan stopword bahasa Indonesia
vectorizer = TfidfVectorizer(stop_words=['pantai', 'gunung', 'wisata', 'kuliner', 'air', 'budaya'])
tfidf_matrix = vectorizer.fit_transform(df['description'])

# Fungsi untuk rekomendasi berbasis kata kunci
def get_recommendations(keyword):
    query_vec = vectorizer.transform([keyword])
    cosine_sim = linear_kernel(query_vec, tfidf_matrix).flatten()
    recommended_indices = cosine_sim.argsort()[-5:][::-1]  # Top 5 destinasi
    return df.iloc[recommended_indices]

# Evaluasi menggunakan MAE dan RMSE
y_true = df['final_score']
y_pred = np.mean(df['final_score'])

# Menghitung MAE dan RMSE
mae = mean_absolute_error(y_true, np.full_like(y_true, y_pred))
rmse = math.sqrt(mean_squared_error(y_true, np.full_like(y_true, y_pred)))

# Contoh penggunaan
keyword = 'pantai dan budaya'
recommendations = get_recommendations(keyword)
print("Rekomendasi untuk kata kunci '{}':".format(keyword))
print(recommendations[['destination', 'final_score']])

print("\nMean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

# Contoh confusion matrix (asumsikan tujuan biner, misalnya 1 = 'direkomendasikan', 0 = 'tidak direkomendasikan')
y_true_binary = np.where(df['final_score'] > 0.5, 1, 0)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)
conf_matrix = confusion_matrix(y_true_binary, np.full_like(y_true_binary, y_pred_binary))

print("\nConfusion Matrix:")
print(conf_matrix)


Rekomendasi untuk kata kunci 'pantai dan budaya':
  destination  final_score
4  Raja Ampat     0.400000
3     Bandung     0.280000
2  Yogyakarta     0.746667
1      Lombok     0.473333
0        Bali     0.833333

Mean Absolute Error (MAE): 0.19466666666666663
Root Mean Squared Error (RMSE): 0.20984650474943714

Confusion Matrix:
[[0 3]
 [0 2]]


# Mencoba dengan Data Eco_Place.csv

In [69]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import confusion_matrix, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler

In [70]:
Data=pd.read_csv('eco_place.csv')

In [71]:
Data

,place_id,place_name,place_description,category,city,price,rating,description_location,place_img,gallery_photo_img1,gallery_photo_img2,gallery_photo_img3,place_map
0,1,Taman Nasional Gunung Leuser,Taman Nasional Gunung Leuser adalah salah satu...,"Budaya,Taman Nasional",Aceh,25000,4.5,"Barisan mountain range, Aceh 24653",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Taman+Nasio...
1,2,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,Desa Wisata,Bali,10000,4.5,"Munduk, Banjar, Kabupaten Buleleng, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://goo.gl/maps/LyeJ2mAeFGysTE9v9
2,3,Desa Wisata Penglipuran,Desa Wisata Penglipuran adalah sebuah desa wis...,"Budaya,Desa Wisata",Bali,25000,4.8,"Jl. Penglipuran, Kubu, Kec. Bangli, Kabupaten ...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Desa+Wisata...
3,4,Taman Nasional Bali Barat,Taman Nasional Bali Barat adalah kawasan konse...,"Taman Nasional,Cagar Alam",Bali,15000,4.5,"Sumber Klampok, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Taman+Nasio...
4,5,Bukit Jamur,Bukit Jamur Ciwidey adalah satu dari sekian ba...,Cagar Alam,Bandung,12000,4.2,"Sugihmukti, Kec. Pasirjambu, Kabupaten Bandung...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Bukit+Jamur
...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,178,Studio Alam Gamplong,Studio Gamplong atau Studio Alam Gamplong adal...,Taman Hiburan,Yogyakarta,10000,4.4,"Gamplong 1, Dukuh, Sumberrahayu, Kec. Moyudan,...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,NaN,https://www.google.com/maps/search/Studio+Alam...
178,179,Watu Goyang,Watu Goyang adalah sebuah fenomena geologis ya...,"Budaya,Cagar Alam",Yogyakarta,2500,4.4,"Cempluk, Jl. Watu Goyang No.1, Griloyo, Mangun...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Watu+Goyang
179,180,Watu Lumbung,Letak Kampung Edukasi Watu Lumbung yang berada...,Cagar Alam,Yogyakarta,5000,4.3,"Area Hutan, Balong, Kec. Girisubo, Kabupaten G...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Watu+Lumbung
180,181,Wisata Alam Kalibiru,Kalibiru sebenarnya adalah sebuah nama hutan y...,"Budaya,Cagar Alam",Yogyakarta,20000,4.4,"Jalan Waduk Sermo, Kalibiru, Hargowilis, Kec. ...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Wisata+Alam...


In [72]:
# Melihat kolom yang ada
Data.columns

Index(['place_id', 'place_name', 'place_description', 'category', 'city',
       'price', 'rating', 'description_location', 'place_img',
       'gallery_photo_img1', 'gallery_photo_img2', 'gallery_photo_img3',
       'place_map'],
      dtype='object')

In [80]:
# Hanya kolom place_name, place_description, price, dan rating yang digunakan
Data2=Data[["place_name","place_description","rating","price"]]
Data2

,place_name,place_description,rating,price
0,Taman Nasional Gunung Leuser,Taman Nasional Gunung Leuser adalah salah satu...,4.5,25000
1,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,4.5,10000
2,Desa Wisata Penglipuran,Desa Wisata Penglipuran adalah sebuah desa wis...,4.8,25000
3,Taman Nasional Bali Barat,Taman Nasional Bali Barat adalah kawasan konse...,4.5,15000
4,Bukit Jamur,Bukit Jamur Ciwidey adalah satu dari sekian ba...,4.2,12000
...,...,...,...,...
177,Studio Alam Gamplong,Studio Gamplong atau Studio Alam Gamplong adal...,4.4,10000
178,Watu Goyang,Watu Goyang adalah sebuah fenomena geologis ya...,4.4,2500
179,Watu Lumbung,Letak Kampung Edukasi Watu Lumbung yang berada...,4.3,5000
180,Wisata Alam Kalibiru,Kalibiru sebenarnya adalah sebuah nama hutan y...,4.4,20000


In [81]:
# Merename Nama Kolom
df2=Data2.rename({"place_name":"destination","place_description":"description"},axis=1)
df2

,destination,description,rating,price
0,Taman Nasional Gunung Leuser,Taman Nasional Gunung Leuser adalah salah satu...,4.5,25000
1,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,4.5,10000
2,Desa Wisata Penglipuran,Desa Wisata Penglipuran adalah sebuah desa wis...,4.8,25000
3,Taman Nasional Bali Barat,Taman Nasional Bali Barat adalah kawasan konse...,4.5,15000
4,Bukit Jamur,Bukit Jamur Ciwidey adalah satu dari sekian ba...,4.2,12000
...,...,...,...,...
177,Studio Alam Gamplong,Studio Gamplong atau Studio Alam Gamplong adal...,4.4,10000
178,Watu Goyang,Watu Goyang adalah sebuah fenomena geologis ya...,4.4,2500
179,Watu Lumbung,Letak Kampung Edukasi Watu Lumbung yang berada...,4.3,5000
180,Wisata Alam Kalibiru,Kalibiru sebenarnya adalah sebuah nama hutan y...,4.4,20000


In [82]:
# Normalisasi kriteria numerik (rating, harga)
scaler = MinMaxScaler()
df[['rating_norm', 'price_norm']] = scaler.fit_transform(df[['rating', 'price']])

# Bobot untuk masing-masing kriteria
weights = {'rating': 0.4, 'price': 0.2}

# Menghitung skor akhir berdasarkan bobot
df['final_score'] = (
    weights['rating'] * df['rating_norm'] +
    weights['price'] * (1 - df['price_norm']) # Harga lebih murah lebih ba
)

# Vektorisasi teks menggunakan TF-IDF dan stopword bahasa Indonesia
vectorizer = TfidfVectorizer(stop_words=['pantai', 'gunung', 'wisata', 'kuliner', 'air', 'budaya'])
tfidf_matrix = vectorizer.fit_transform(df['description'])

# Fungsi untuk rekomendasi berbasis kata kunci
def get_recommendations(keyword):
    query_vec = vectorizer.transform([keyword])
    cosine_sim = linear_kernel(query_vec, tfidf_matrix).flatten()
    recommended_indices = cosine_sim.argsort()[-5:][::-1]  # Top 5 destinasi
    return df.iloc[recommended_indices]

# Evaluasi menggunakan MAE dan RMSE
y_true = df['final_score']
y_pred = np.mean(df['final_score'])

# Menghitung MAE dan RMSE
mae = mean_absolute_error(y_true, np.full_like(y_true, y_pred))
rmse = math.sqrt(mean_squared_error(y_true, np.full_like(y_true, y_pred)))

# Contoh penggunaan
keyword = 'pantai dan budaya'
recommendations = get_recommendations(keyword)
print("Rekomendasi untuk kata kunci '{}':".format(keyword))
print(recommendations[['destination', 'final_score']])

print("\nMean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

# Contoh confusion matrix (asumsikan tujuan biner, misalnya 1 = 'direkomendasikan', 0 = 'tidak direkomendasikan')
y_true_binary = np.where(df['final_score'] > 0.5, 1, 0)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)
conf_matrix = confusion_matrix(y_true_binary, np.full_like(y_true_binary, y_pred_binary))

print("\nConfusion Matrix:")
print(conf_matrix)

Rekomendasi untuk kata kunci 'pantai dan budaya':
                          destination  final_score
96          Taman Nasional Tesso Nilo     0.346667
123      Taman Wisata Alam Situgunung     0.338889
122  Taman Wisata Alam Curug Cipendok     0.448000
49     Taman Wisata Alam Riung Gunung     0.473889
38       Taman Wisata Alam Ranca Upas     0.469444

Mean Absolute Error (MAE): 0.04208304150062393
Root Mean Squared Error (RMSE): 0.056027817793564465

Confusion Matrix:
[[161   0]
 [ 21   0]]


# Menggunakan Dataset Wisata Batu

In [84]:
Data=pd.read_excel("Dataset_227(10F).xlsx")
Data

,jenis_kelamin,umur,pekerjaan,hobi,tujuan_berwisata,status_perkawinan,daerah_asal,teman_perjalanan,pendidikan_terakhir,repetition,...,Y5,Y6,Y7,Y8,Y9,Y10,Y11,Y12,Y13,Y14
0,Laki-Laki,12-25 tahun,Pelajar/Mahasiswa,Olahraga,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Pasangan,SMA/SMK,Pernah,...,1,0,0,0,0,0,0,0,0,0
1,Perempuan,12-25 tahun,Wiraswasta,Traveling,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Teman,SMA/SMK,Belum Pernah,...,1,0,0,1,0,0,0,0,0,0
2,Perempuan,12-25 tahun,Pelajar/Mahasiswa,Traveling,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Keluarga,SMA/SMK,Pernah,...,1,1,0,1,0,0,0,1,0,0
3,Perempuan,12-25 tahun,Pelajar/Mahasiswa,Traveling,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Teman,SMA/SMK,Belum Pernah,...,1,1,0,1,0,1,0,0,0,0
4,Laki-Laki,12-25 tahun,Pelajar/Mahasiswa,Olahraga,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Teman,SMA/SMK,Pernah,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Perempuan,26-45 tahun,Lainnya,Traveling,Rekreasi/Liburan,Menikah,Kota Batu,Keluarga,S1/D1/D2/D3,Pernah,...,1,0,1,0,0,0,0,0,0,0
223,Laki-Laki,26-45 tahun,Lainnya,Berenang,Rekreasi/Liburan,Menikah,Kota Batu,Keluarga,SMA/SMK,Pernah,...,0,1,0,0,0,0,0,0,0,0
224,Laki-Laki,26-45 tahun,Wiraswasta,Traveling,Rekreasi/Liburan,Menikah,Kota Batu,Pasangan,S1/D1/D2/D3,Pernah,...,0,0,0,0,0,0,0,0,0,1
225,Laki-Laki,26-45 tahun,Wiraswasta,Traveling,Rekreasi/Liburan,Menikah,Kota Batu,Keluarga,S1/D1/D2/D3,Pernah,...,1,0,0,0,0,0,0,0,0,0


In [85]:
Data.columns

Index(['jenis_kelamin', 'umur', 'pekerjaan', 'hobi', 'tujuan_berwisata',
       'status_perkawinan', 'daerah_asal', 'teman_perjalanan',
       'pendidikan_terakhir', 'repetition', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6',
       'Y7', 'Y8', 'Y9', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14'],
      dtype='object')